# User Generation

Untuk Population Policy, mengalokasikan sumber beban kerja (*workload generators*), seperti sensor atau pengguna, ke node dalam topologi. Kebijakan ini juga mendefinisikan pesan apa yang mereka hasilkan dan seberapa sering (menggunakan distribusi waktu)

In [6]:
import networkx as nx
import operator
import json
import random

# KONFIGURASI
FUNC_NETWORK_GENERATION = "nx.barabasi_albert_graph(n=100, m=2)"
FUNC_NODE_RESOURCES = "random.randint(10,25)" # MB RAM
FUNC_NODE_SPEED = "random.randint(100,1000)" # INST / MS
FUNC_NODE_TB = "random.uniform(0.2,100)" # TB
FUNC_PROPAGATION_TIME = "random.randint(3,5)" # MS
FUNC_BANDWIDTH = "random.randint(6*10**6,6*10**7)" # BIT / S
PERCENTAGE_GATEWAYS = 0.25
CLOUD_CAPACITY = 9999999999999999 # MB RAM
CLOUD_SPEED = 10000 # INSTR X MS
CLOUD_PR = 1 # MS
CLOUD_BW = 125000 # BYTES / MS -> 1000 Mbits/s

# Setup topology and gateway devices
# NETWORK GENERATION
G = eval(FUNC_NETWORK_GENERATION)

# nx.draw(G)

# Declare list dari semua perangkat (fog node)
devices = list()

# Assign value ke node (IPT, RAM, TB) dan edge (PR, BW)
node_resources = {}
node_free_resources = {}
node_speed = {}
node_storage = {}
for i in G.nodes:
    node_resources[i] = eval(FUNC_NODE_RESOURCES)
    node_speed[i] = eval(FUNC_NODE_SPEED)
    node_storage[i] = eval(FUNC_NODE_TB)

for e in G.edges:
    G[e[0]][e[1]]["PR"] = eval(FUNC_PROPAGATION_TIME)
    G[e[0]][e[1]]["BW"] = eval(FUNC_BANDWIDTH)

# JSON EXPORT
net_json = {}

for i in G.nodes:
    my_node = {}
    my_node["id"] = i
    my_node["RAM"] = node_resources[i]
    my_node["IPT"] = node_speed[i]
    my_node["TB"] = node_storage[i]
    devices.append(my_node)

my_edges = list()
for e in G.edges:
    my_link = {}
    my_link["s"] = e[0]
    my_link["d"] = e[1]
    my_link["PR"] = G[e[0]][e[1]]["PR"]
    my_link["BW"] = G[e[0]][e[1]]["BW"]
    my_edges.append(my_link)

# Mencari nilai betweenness centrality dari setiap node
centrality_values_no_ordered = nx.betweenness_centrality(
    G, weight="weight"
)

# Urutkan betweeness centrality dari yang tertinggi
centrality_values = sorted(
    centrality_values_no_ordered.items(),
    key=operator.itemgetter(1),
    reverse=True,
)
# centrality_values = [[key_node, centrality_value], ...]

gateway_devices = set()
cloud_gateway_devices = set()

# ambil nilai betweeness centrality tertinggi
highest_centrality = centrality_values[0][1]

# TODO - Atur Cloud-Fog-Gateway menjadi 5% alih-alih hanya ambil betweeness centrality tertinggi
# cek device mana yang punya nilai centrality tertinggi
# lalu tambahkan device tersebut ke cloud_gateway_devices
for device in centrality_values:
    if device[1] == highest_centrality:
        cloud_gateway_devices.add(device[0])

# mencari indeks awal untuk gateway device
initial_idx = int((1 - PERCENTAGE_GATEWAYS) * len(G.nodes))

# karena centrality sudah diurutkan dari tertinggi ke terendah
# maka indeks awal (misal 75) sampai jumlah node (misal 100)
# adalah node dengan centrality terendah yang akan menjadi gateway device
for id_dev in range(initial_idx, len(G.nodes)):
    gateway_devices.add(centrality_values[id_dev][0])

# menambahkan 1 node sebagai cloud
# print(f"sebelum: {devices}")

cloud_id = len(G.nodes)
my_node = {}
my_node["id"] = cloud_id
my_node["RAM"] = CLOUD_CAPACITY
my_node["IPT"] = CLOUD_SPEED
my_node["type"] = "CLOUD"
devices.append(my_node)

# menambahkan edge dari cloud gateway
for cloud_gateway in cloud_gateway_devices:
    my_link = {}
    my_link["s"] = cloud_gateway
    my_link["d"] = cloud_id
    my_link["PR"] = CLOUD_PR
    my_link["BW"] = CLOUD_BW

    my_edges.append(my_link)

# convert ke JSON
net_json["entity"] = devices
net_json["link"] = my_edges

file = open("./generated/networkDefinition.json", "w")
file.write(json.dumps(net_json))
file.close()

In [7]:
# USER GENERATION

# KONFIGURASI
# Jumlah aplikasi
TOTAL_APP_NUMBER = 20
# App popularity: threshold yang menentukan probabilitas setiap app akan ada workload (0 - 0.25)
FUNC_REQUEST_PROB = "random.random()/4"
# Request rate: seberapa sering (1 req/ 200-1000MS)
FUNC_USER_REQ_RAT = "random.randint(200,1000)"

user_json = {}

my_users = list()
app_requests = list()

for i in range(0, TOTAL_APP_NUMBER):
    user_request_list = set()
    prob_of_requested = eval(FUNC_REQUEST_PROB)
    at_least_one_allocated = False

    for j in gateway_devices:
        if random.random() < prob_of_requested:
            my_one_user = {}
            my_one_user["app"] = str(i)
            my_one_user["message"] = "M.USER.APP." + str(i)
            my_one_user["id_resource"] = j
            my_one_user["lambda"] = eval(FUNC_USER_REQ_RAT)
            user_request_list.add(j)
            my_users.append(my_one_user)
            at_least_one_allocated = True

    # jika dalam loop pertama tidak menempatkan satu pun workload pada aplikasi
    if not at_least_one_allocated:
        j = random.randint(0, len(gateway_devices) - 1)
        my_one_user = {}
        my_one_user["app"] = str(i)
        my_one_user["message"] = "M.USER.APP." + str(i)
        my_one_user["id_resource"] = j
        my_one_user["lambda"] = eval(FUNC_USER_REQ_RAT)
        user_request_list.add(j)
        my_users.append(my_one_user)

    app_requests.append(user_request_list)

user_json["sources"] = my_users
file = open("./generated/usersDefinition.json", "w")
file.write(json.dumps(user_json))
file.close()
